# Set up Spark environment

In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
        .master('local[*]')
        .appName('Intro to Spark')
        .config('spark.ui.port', '4050')
        .getOrCreate())
spark

# Exercises

Before starting with the exercises we need to upload the files `people.csv` and `department.csv` in the root folder. These files are necessary to create the dataframes `people_df` and `department_df`, respectively.

In [ ]:
people_df = spark.read.csv("people.csv", header=True, inferSchema=True)
assert people_df.count() == 1000
print("File people.csv correctly uploaded!")

In [ ]:
people_df.show()

In [ ]:
people_df.printSchema()

In [ ]:
department_df = spark.read.csv("department.csv", header=True, inferSchema=True)
assert department_df.count() == 900
print("File department.csv correctly uploaded!")

In [ ]:
department_df.show()

In [ ]:
department_df.printSchema()

In [8]:
from pyspark.sql.functions import *
from pyspark.sql import *

## Exercise 1

Create the dataframe `female_df` from the dataframe `people_df`, by selecting the columns `id` and `gender` and filtering the gender column for the value `Female`.

In [9]:
female_df = people_df.select('id', 'gender').filter(col("gender") == "Female")

In [ ]:
assert female_df.schema.fieldNames() == ['id', 'gender'] 
assert female_df.count() == 455
print("Exercise 1 passed!")

## Exercise 2

Create the dataframe`full_address_df` from the dataframe `people_df`, by selecting the columns `city` and `country` and adding a new column `full_address`, which has to be created by concatenating the columns `city` and `country`, separated by a blank space.

In [11]:
full_address_df = people_df.select(col('city'), col('country'), concat_ws(' ', col('city'), col('country')).alias('full_address'))

In [ ]:
assert full_address_df.schema.fieldNames() == ['city', 'country', 'full_address'] 
assert full_address_df.filter(col('city') == 'Haapsalu').select('full_address').collect()[0][0] == 'Haapsalu Estonia'
print("Exercise 2 passed!")

## Exercise 3

Create the dataframe `ethereum_df` from the dataframe `people_df` by selecting the column `ethereum_address` and applying a filter which keeps only the rows that start with `0x1`, `0x2` and `0x3`.

Then add a new column `newCol`, with these conditions:

1.   If `ethereum_address` starts with `0x1` put `a`
2.   Se `ethereum_address` starts with `0x2` put `b`
3.   Se `ethereum_address` starts with `0x3` put `c`

In the `ethereum_df` dataframe must appear only the columns `ethereum_address` and `newCol`.

In [13]:
ethereum_df = (people_df.select('ethereum_address', substring(col('ethereum_address'), 0, 3).alias('temp')) 
                    .filter(col('temp').isin('0x1', '0x2', '0x3')) 
                    .withColumn('newCol', when(col('temp') == '0x1', 'a').when(col('temp') == '0x2', 'b').otherwise('c')) 
                    .drop('temp'))

In [ ]:
assert ethereum_df.schema.fieldNames() == ['ethereum_address', 'newCol'] 
assert ethereum_df.count() == 183
row = ethereum_df.filter(col('ethereum_address') == '0x1b907715611571700163387611121552d1ad1c9e').collect()[0]
assert row['newCol'] == 'a'
print("Exercise 3 passed!")

## Exercise 4

Create a new dataframe `sql_ethereum_df` from the dataframe `people_df` by creating a column `newCol` with the same conditions of the previous exercise, 

then we need to perform a substring on `newCol` and apply a filter that keeps only the record equal to `0x1`, `0x2` and `0x3`.

Then we need to count the rows for each value of `newCol`` (`0x1`, `0x2` and `0x3`) in a new column `count`.

For this exercise we will make use of `Spark SQL`.

The dataframe `sql_ethereum_df` must have only the columns `newCol` and `count`.

In [39]:
people_df.createOrReplaceTempView('people_df')
sql_ethereum_df = (spark.sql("""
                      SELECT 
                          left(ethereum_address,3) AS newCol,
                          count(*) AS count
                      FROM people_df
                      WHERE left(ethereum_address,3) IN ('0x1','0x2','0x3')
                      GROUP BY newCol
                  """))

In [ ]:
assert sql_ethereum_df.schema.fieldNames() == ['newCol', 'count'] 
assert sql_ethereum_df.count() == 3
row = sql_ethereum_df.filter(col('newCol') == '0x1').collect()[0]
assert row['count'] == 66
print("Exercise 4 passed!")

## Exercise 5

Let's create a new dataframe `male_df` from the `people_df` dataframe by filtering the records with the column `gender` equal to `Male`.

Then let's create a new column `count` which contains the counts of the records per `country` and at finally order the rows by `count` in descending order.

In [63]:
male_df = (people_df.filter(col('gender') == 'Male')
                .groupBy('country')
                .agg(count("id").alias("count"))
                .orderBy("count", ascending = False))

In [ ]:
assert male_df.schema.fieldNames() == ['country', 'count'] 
row_max =  male_df.collect()[0]
assert row_max['country'] == 'China'
assert row_max['count'] == 89
print("Exercise 5 passed!")

## Exercise 6

Create a new dataframe `people_dept_df` by performing an inner join on the dataframes `people_df` and `department_df` on the column `id`.

In [65]:
people_dept_df = people_df.join(department_df, 'id', 'inner')

In [ ]:
assert people_dept_df.count() == 900
assert people_dept_df.schema.fieldNames() == ['id', 'first_name', 'last_name', 'email', 'gender', 'ip_address', 'city', 'country', 'ethereum_address', 'department']
print("Exercise 6 passed!")

## Exercise 7

Create a new dataframe `count_people_dept_df` from the `people_dept_df` by performing a group by on the `department` column and counting the number of people for each department.

The column created has to be called `count`.

Then let's apply a filter and keep only the records with a count greater then 200.

In [67]:
count_people_dept_df = people_dept_df.groupBy('department').agg(count("id").alias("count")).filter(col('count') > 200)

In [ ]:
assert count_people_dept_df.count() == 1
assert count_people_dept_df.schema.fieldNames() == ['department', 'count']
row = count_people_dept_df.collect()[0]
assert row['count'] == 203
print("Exercise 7 passed!")

## Exercise 8

Let's repeat the steps of the previous exercise by using `Spark SQL` and creating the `sql_count_people_dept_df` dataframe.

In [77]:
people_dept_df.createOrReplaceTempView("people_dept_df")
sql_count_people_dept_df = (spark.sql("""
                              SELECT
                                  department,
                                  count(*) AS count
                              FROM people_dept_df
                              GROUP BY department
                              HAVING count > 200
                            """))

In [ ]:
assert sql_count_people_dept_df.count() == 1
assert sql_count_people_dept_df.schema.fieldNames() == ['department', 'count']
row = sql_count_people_dept_df.collect()[0]
assert row['count'] == 203
print("Exercise 8 passed!")

# Exercise 9

Let's create the dataframe `ip_address_per_country_df` from the `people_df` dataframe by applying a filter on the `ip_address` column and keeping only the ip addresses which start with `21`.

Afterwards let's calculate which country has the greatest number of records. 

Let's make use of `pyspark.sql.Column.like`

In [97]:
ip_address_per_country_df = (people_df.filter(col('ip_address').like('21%'))
        .groupBy('country')
        .agg(count("ip_address").alias("count"))
        .orderBy('count', ascending = False)
        .limit(1))                                    

In [ ]:
assert ip_address_per_country_df.schema.fieldNames() == ['country', 'count']
row = ip_address_per_country_df.filter(col('country') == 'Indonesia').collect()[0]
assert row['count'] == 7
print("Exercise 9 passed!")

## Exercise 10

Let's create the `email_per_gender_df` dataframe from the `people_df` dataframe by applying a filter on the `email` column and keeping only the records which end with `.com`.

Then let's group by the column `gender` and perform a count thus creating a column called `count`.

Finally let's keep only the row with the greatest count.

Let's use `pyspark.sql.Column.rlike`

In [ ]:
email_per_gender_df = (people_df.filter(col('email').rlike('.com$'))
                        .groupBy('gender')
                        .agg(count("ip_address").alias("count"))
                        .orderBy('count', ascending = False)
                        .limit(1))   

In [ ]:
assert email_per_gender_df.count() == 1
assert email_per_gender_df.schema.fieldNames() == ['gender', 'count']
row = email_per_gender_df.collect()[0]
assert row['gender'] == 'Female'
assert row['count'] == 275
print("Exercise 10 passed!")